In [1]:
import pandas as pd
import operator
import numpy as np
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from scipy.stats import norm
from numpy.random import choice
from math import sqrt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix

#sns.set_style('darkgrid')
np.random.seed(500)

### Our path to all the dataset genrated or loaded will be specified here!

In [2]:
#path = '/Users/mosesopenja/Documents/Fall2020/Machine-learning/assignment2/dataset/hwk2_datasets/'
#path_output = '/Users/mosesopenja/Documents/Fall2020/Machine-learning/assignment2/dataset/generated/'

path = './inputs/'
path_output = './'

## Linear Classification and Nearest Neighbor Classification



### Question 1.

1. You will use a synthetic data set for the classification task that you’ll generate yourself.
Generate two classes with 20 features each. Each class is given by a multivariate Gaussian
distribution, with both classes sharing the same covariance matrix. You are provided
with the mean vectors (DS1-m0 for mean vector of negative class and DS1-m1 for mean
vector of positive class) and the covariance matrix (DS1-cov). Generate 2000 examples
for each class, and label the data to be positive if they came from the Gaussian with
mean m1 and negative if they came from the Gaussian with mean m0. Randomly pick
(without replacement) 20% of each class (i.e., 400 data points per class) as test set, 20%
of each class (i.e., 400 data points per class) as validation set set and train the classifiers
on the remaining 60% data. When you report performance results, it should be on the
test set.  Call this dataset as DS1, and submit it with your code. Follow the instructions
from Assignment 1 for data submission format.

#### Lets start by loading the neccasary dataset DS1_m_0.txt, DS1_m_1.txt, and DS1_Cov.txt

In [3]:
DS1_m_0 = np.genfromtxt(path+'DS1_m_0.txt', delimiter=',')[:-1]# load the mean0 and remove the last emply string
DS1_m_1 = np.genfromtxt(path+'DS1_m_1.txt', delimiter=',')[:-1]# load the mean1 and remove the last emply string
DS1_Cov = np.genfromtxt(path+'DS1_Cov.txt', delimiter=',')[:,:-1] # load the covariace and remove the last emply string
DS1_m_1

array([2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2.])

In [4]:
def append_target_value(class_array, target_array):
    return np.append(class_array,target_array.reshape(-1,1), axis=1)
def append_more_rows(class_array_1, class_array_2):
    return np.append(class_array_1,class_array_2, axis=0)

In [5]:
neg = np.random.multivariate_normal(DS1_m_0, DS1_Cov, 2000)
pos = np.random.multivariate_normal(DS1_m_1, DS1_Cov, 2000)

In [6]:
class_negative = append_target_value(neg, np.zeros((2000,1), dtype=int)) # append the targets 0 for negative class
class_positive = append_target_value(pos, np.ones((2000,1), dtype=int)) # append the targets 1 for positive class

In [7]:
#Define the function to split the data into train, test validate as 600,200,200
def train_test_valid_split(class_data):
    train, test = train_test_split(class_data, test_size=0.2, random_state=42)
    train, valid = train_test_split(train, test_size=0.25, random_state=42)
    return train, test, valid

In [8]:
# we can use the function defined in upper scope to split our data
train_1, test_1, valid_1 = train_test_valid_split(class_negative)
train_2, test_2, valid_2 = train_test_valid_split(class_positive)

In [9]:
#lets join the dataset of class 1 and two
train = np.append(train_1, train_2, axis=0)
test = np.append(test_1, test_2, axis=0)
valid = np.append(valid_1, valid_2, axis=0)

In [10]:
# lets check the shape of our final dataset:
print(train.shape,test.shape, valid.shape )

(2400, 21) (800, 21) (800, 21)


In [11]:
#Dump the dataset as csv files with the format defined in the TP intructions:
np.savetxt(path_output+"DS1_train.csv", train, delimiter=",")
np.savetxt(path_output+"DS1_test.csv", test, delimiter=",")
np.savetxt(path_output+"DS1_valid.csv", valid, delimiter=",")

#### Lets implement K-NN clafication algorithm  that I can use to test the performance of the dataset generated

The algorithm expect the two params x_train and y_train to fit the dataset and test set to predict!

In [12]:
# defind the function for euclidean distance given two vectors
def euclidean_distance(x1, x2):
    return np.sqrt(np.sum((x1-x2)**2))

class KNN(): # Class KNearestNeighbours goes here..!
    def __init__(self, K=5): # we can set the default neightbours to 5
        self.K = K
    def fit(self, x_train, y_train): # fit function expect the vectors and the target
        self.X_train = x_train
        self.Y_train = y_train
    def predict(self, X_test): 
        # list to store all our predictions
        predictions = []
        # loop over all observations in the test set
        for i in range(len(X_test)):            
            # find the value of distance between the test point and all other points in the training set
            dist = np.array([euclidean_distance(X_test[i], x_t) for x_t in self.X_train])
            # we can now sort the distances and return the indices of K neighbors
            dist_sorted = dist.argsort()[:self.K]
            # pick the neighbors
            neighbors_count = {}
            # for each neighbor find the class
            for idx in dist_sorted:
                if self.Y_train[idx][0] in neighbors_count:
                    neighbors_count[self.Y_train[idx][0]] += 1
                else:
                    neighbors_count[self.Y_train[idx][0]] = 1
            # sort the neighbours_count
            sorted_neighbors_count = sorted(neighbors_count.items(), key=operator.itemgetter(1), reverse=True)
            # lets add/ append our class label to the predict list
            predictions.append(sorted_neighbors_count[0][0])
        return predictions

In [54]:
# lets extract the target from the train and test
x_train = train[0:,0:20]
y_train = train[0:,20:]
x_test = test[0:,0:20]
y_test = test[0:,20:]
x_valid = test[0:,0:20]
y_valid = test[0:,20:]

In [14]:
'''testing the performance of our dataset with KNN classifier defined in the upper scope
   --- we can use K=5, for now -----'''
knn_model = KNN(K=5)
knn_model.fit(x_train,y_train)

In [55]:
def get_scores_metrics(gda_pred):
    accuracy = metrics.accuracy_score(y_test, gda_pred)          # accuracy: (tp + tn) / (p + n)
    precision = metrics.precision_score(y_test, gda_pred)        # precision tp / (tp + fp)
    recall = metrics.recall_score(y_test, gda_pred)              # recall: tp / (tp + fn)
    f1 = metrics.f1_score(y_test, gda_pred)                      # f1: 2 tp / (2 tp + fp + fn)
    return precision, recall, f1, accuracy

In [16]:
# Lets check the performance of our dataset!
pred = knn_model.predict(x_test)# NOTE: predict step in KNN is much slower than the train step

In [17]:
precision, recall, f1, accuracy = get_scores_metrics(pred)

In [18]:
#print the results
print('Accuracy: %f' % accuracy)
print('Precision: %f' % precision)
print('Recall: %f' % recall)
print('F1 score: %f' % f1)

Accuracy: 0.578750
Precision: 0.583113
Recall: 0.552500
F1 score: 0.567394


## Question 2.

2. We first consider the GDA model as seen in class: given the class variable, the data are assumed to be Gaussians with different means for different classes but with the same covariance matrix. This model can formally be specified as follows:

 1. Estimate the parameters of the GDA model using the maximum likelihood approach.
 
 a). For DS1, report the best fit accuracy, precision, recall and F-measure achieved
by the classifier.

In [19]:
#lets implement a GDA classifier

class GDAClassification:

    def fit(self, X, y):
        self.labels, self.class_priors = np.unique(y, return_counts=True)
        self.class_priors = self.class_priors / y.shape[0]
        
        self.Train_Data = X
        self.Train_Label = y
        
        print(self.labels)
        
        self.dict_data = {}
        self.dict_data_count = {}
        for (data,label) in zip(self.Train_Data,self.Train_Label):
 
            if label[0] in self.dict_data.keys():
                temp = self.dict_data.get(label[0])
                temp_count = self.dict_data_count.get(label[0])
                temp_count += 1
                temp.append(list(data))
                self.dict_data[label[0]] = temp
                self.dict_data_count[label[0]] = temp_count
            else:
                self.dict_data[label[0]] = [list(data)]
                self.dict_data_count[label[0]] = 1
            
                    
        # Binomial probability goas here:
        row,col = np.shape(X)
        self.binomial_dict = {}
        for key, val in self.dict_data_count.items():
            self.binomial_dict[key] = val*1.0/row
        
        dict_data_array = {}
        data_sum_dict = {}
        for k, v in self.dict_data.items():
            dict_data_array[k] = np.array(v)
            data_sum_dict[k]= np.sum(np.array(v),0)
        
        self.mu_data = {}
        for k, v in data_sum_dict.items():
            self.mu_data[k] = v*1.0/self.dict_data_count[k]
            
        dict_data_array2 = {}    
        for k, v in dict_data_array.items():
            dict_data_array2[k] = v-self.mu_data[k]
        
        self.sigma = []
        
        for k,v in dict_data_array2.items():
            for deta in v:
                deta = deta.reshape(1,col)
                ans = deta.T.dot(deta)
                self.sigma.append(ans)

        self.sigma = np.array(self.sigma)
        #print(np.shape(self.sigma))
        self.sigma = np.sum(self.sigma,0)
        self.sigma = self.sigma/row
        
        
        for k, v in self.mu_data.items():
            self.mu_data[k] = v.reshape(1,col)
            print("Mu_{}: {}".format(k,v.reshape(1,col)))
        print(self.sigma)
        
    def gaussian(self, x, mean, cov):
        
        dim = np.shape(cov)[0]
        # Cov measures of the determinant is zero
        covdet = np.linalg.det(cov + np.eye(dim) * 0.001)
        covinv = np.linalg.inv(cov + np.eye(dim) * 0.001)
        xdiff = (x - mean).reshape((1, dim))
        # Probability Density
        p_density = 1.0 / (np.power(np.power(2 * np.pi, dim) * np.abs(covdet), 0.5)) * \
               np.exp(-0.5 * xdiff.dot(covinv).dot(xdiff.T))[0][0]
        return p_density

    def predict(self,test_data):
        predict_label = []
        for data in test_data:
            max_label = 0
            max_likehood = 0
            for lab in self.labels:
                likelihood = self.gaussian(data,self.mu_data[lab],self.sigma)
                if likelihood >= max_likehood:
                    max_likehood = likelihood
                    max_label = lab
            predict_label.append(max_label)
        return predict_label

In [20]:
# train and test the performance of our Dataset DS1 with GDA above
gda = GDAClassification()
gda.fit(x_train,y_train)
gda_pred = gda.predict(x_test)

[0. 1.]
Mu_0.0: [[1.35464974 1.31507699 1.3314178  1.2815658  1.32796116 1.3255876
  1.2801238  1.30097595 1.29455227 1.19711815 1.33164364 1.30814516
  1.24153705 1.30350525 1.36083324 1.34496766 1.37031654 1.3047932
  1.34410696 1.29102756]]
Mu_1.0: [[2.11957295 2.09784241 2.0733269  2.08536989 2.07177151 2.09712021
  2.08439585 2.12640347 2.12214562 2.16672049 2.05441559 2.12151196
  2.13261964 2.13715674 2.07409241 2.12984784 2.0945371  2.12444338
  2.08053467 2.06898178]]
[[8.05737947 5.49391003 6.13895501 5.28240859 5.93593272 6.08110628
  4.5398847  5.3466409  5.05495747 5.19376925 4.07712252 5.21028971
  7.15659755 6.18175617 6.15940356 5.90644234 5.91970428 5.8573333
  5.63549282 6.12416714]
 [5.49391003 6.91870004 5.35184565 4.3595837  5.48277723 5.5054034
  4.21500962 3.77226978 4.20085728 5.00453663 3.42320773 4.54801931
  5.84677566 5.19180032 5.44611408 5.10122074 5.72850822 5.2261086
  5.3282477  5.39502424]
 [6.13895501 5.35184565 7.18241428 4.82499666 5.77823116 6.4609

In [21]:
def get_scores_metrics(gda_pred):
    accuracy = metrics.accuracy_score(y_test, gda_pred)          # accuracy: (tp + tn) / (p + n)
    precision = metrics.precision_score(y_test, gda_pred, average='macro')        # precision tp / (tp + fp)
    recall = metrics.recall_score(y_test, gda_pred, average='macro')              # recall: tp / (tp + fn)
    f1 = metrics.f1_score(y_test, gda_pred, average='macro')                      # f1: 2 tp / (2 tp + fp + fn)
    return precision, recall, f1, accuracy

In [22]:
precision, recall, f1, accuracy = get_scores_metrics(gda_pred)

In [23]:
#print the results
print('Accuracy: %f' % accuracy)
print('Precision: %f' % precision)
print('Recall: %f' % recall)
print('F1 score: %f' % f1)

Accuracy: 0.966250
Precision: 0.966276
Recall: 0.966250
F1 score: 0.966250


## Question 3:

3. For DS1, use k-NN to learn a classifier. Repeat the experiment for different values of k
and report the performance for each value. We will compare this non-linear classifier to
the linear approach, and find out how powerful linear classifiers can be.


(a) Does this classifier performs better than GDA or worse? Are there particular values
of k which perform better? Why does this happen ? Use F1-Measure for model
selection.

(b) Report the best fit accuracy, precision, recall and f-measure achieved by this clas-
sifier.


In [24]:
# defind the function for euclidean distance given two vectors
def e_distance(x1, x2): 
        return np.sqrt(np.sum((x1-x2)**2))

In [25]:
class KNN(): # Class KNearestNeighbours goes here..!
    def __init__(self, K=5): # we can set the default neightbours to 5
        self.K = K
    def fit(self, x_train, y_train): # fit function expect the vectors and the target
        self.X_train = x_train
        self.Y_train = y_train
    def predict(self, X_test): 
        # list to store all our predictions
        predictions = []
        # loop over all observations in the test set
        for i in range(len(X_test)):            
            # find the value of distance between the test point and all other points in the training set
            dist = np.array([e_distance(X_test[i], x_t) for x_t in self.X_train])
            # we can now sort the distances and return the indices of K neighbors
            dist_sorted = dist.argsort()[:self.K]
            # pick the neighbors
            neighbors_count = {}
            # for each neighbor find the class
            for idx in dist_sorted:
                if self.Y_train[idx][0] in neighbors_count:
                    neighbors_count[self.Y_train[idx][0]] += 1
                else:
                    neighbors_count[self.Y_train[idx][0]] = 1
            # sort the neighbours_count
            sorted_neighbors_count = sorted(neighbors_count.items(), key=operator.itemgetter(1), reverse=True)
            # lets add/ append our class label to the predict list
            predictions.append(sorted_neighbors_count[0][0])
        return predictions

In [56]:
def get_scores_metrics(gda_pred, y):
    accuracy = metrics.accuracy_score(y, gda_pred)          # accuracy: (tp + tn) / (p + n)
    precision = metrics.precision_score(y, gda_pred)        # precision tp / (tp + fp)
    recall = metrics.recall_score(y, gda_pred)              # recall: tp / (tp + fn)
    f1 = metrics.f1_score(y, gda_pred)                      # f1: 2 tp / (2 tp + fp + fn)
    return precision, recall, f1, accuracy

In [27]:
for k in range (2, 21,2):
    knn_model = KNN(K=k)
    knn_model.fit(x_train,y_train)
    knn_pred = knn_model.predict(x_test)# NOTE: predict step in KNN is much slower than the train step
    precision, recall, f1, accuracy = get_scores_metrics(knn_pred, y_test)
    # Print Performance results for each K value
    print("K=%d precision: %f, recall: %f, f1 score: %f, accuracy: %f"%(k,precision,recall,f1, accuracy))

K=2 precision: 0.542500, recall: 0.542500, f1 score: 0.542500, accuracy: 0.542500
K=4 precision: 0.552500, recall: 0.552500, f1 score: 0.552500, accuracy: 0.552500
K=6 precision: 0.570000, recall: 0.570000, f1 score: 0.570000, accuracy: 0.570000
K=8 precision: 0.553750, recall: 0.553750, f1 score: 0.553750, accuracy: 0.553750
K=10 precision: 0.535000, recall: 0.535000, f1 score: 0.535000, accuracy: 0.535000
K=12 precision: 0.570000, recall: 0.570000, f1 score: 0.570000, accuracy: 0.570000
K=14 precision: 0.565000, recall: 0.565000, f1 score: 0.565000, accuracy: 0.565000
K=16 precision: 0.568750, recall: 0.568750, f1 score: 0.568750, accuracy: 0.568750
K=18 precision: 0.566250, recall: 0.566250, f1 score: 0.566250, accuracy: 0.566250
K=20 precision: 0.573750, recall: 0.573750, f1 score: 0.573750, accuracy: 0.573750


In [28]:
for k in range (15, 51, 5):
    knn_model = KNN(K=k)
    knn_model.fit(x_train,y_train)
    knn_pred = knn_model.predict(x_test)# NOTE: predict step in KNN is much slower than the train step
    precision, recall, f1, accuracy = get_scores_metrics(knn_pred, y_test)
    # Print Performance results for each K value
    print("K=%d precision: %f, recall: %f, f1 score: %f, accuracy: %f"%(k,precision,recall,f1, accuracy))

K=15 precision: 0.562500, recall: 0.562500, f1 score: 0.562500, accuracy: 0.562500
K=20 precision: 0.573750, recall: 0.573750, f1 score: 0.573750, accuracy: 0.573750
K=25 precision: 0.557500, recall: 0.557500, f1 score: 0.557500, accuracy: 0.557500
K=30 precision: 0.556250, recall: 0.556250, f1 score: 0.556250, accuracy: 0.556250
K=35 precision: 0.540000, recall: 0.540000, f1 score: 0.540000, accuracy: 0.540000
K=40 precision: 0.537500, recall: 0.537500, f1 score: 0.537500, accuracy: 0.537500
K=45 precision: 0.542500, recall: 0.542500, f1 score: 0.542500, accuracy: 0.542500
K=50 precision: 0.540000, recall: 0.540000, f1 score: 0.540000, accuracy: 0.540000


In [57]:
#lets check the performance with the validation set too!
# start with K=2 to 20
for k in range (2, 21,2):
    knn_model = KNN(K=k)
    knn_model.fit(x_train,y_train)
    knn_pred = knn_model.predict(x_valid)# NOTE: predict step in KNN is much slower than the train step
    precision, recall, f1, accuracy = get_scores_metrics(knn_pred, y_valid)
    # Print Performance results for each K value
    print("K=%d precision: %f, recall: %f, f1 score: %f, accuracy: %f"%(k,precision,recall,f1, accuracy))

K=2 precision: 0.543590, recall: 0.530000, f1 score: 0.536709, accuracy: 0.542500
K=4 precision: 0.554974, recall: 0.530000, f1 score: 0.542199, accuracy: 0.552500
K=6 precision: 0.572539, recall: 0.552500, f1 score: 0.562341, accuracy: 0.570000
K=8 precision: 0.558266, recall: 0.515000, f1 score: 0.535761, accuracy: 0.553750
K=10 precision: 0.538043, recall: 0.495000, f1 score: 0.515625, accuracy: 0.535000
K=12 precision: 0.576923, recall: 0.525000, f1 score: 0.549738, accuracy: 0.570000
K=14 precision: 0.573446, recall: 0.507500, f1 score: 0.538462, accuracy: 0.565000
K=16 precision: 0.577031, recall: 0.515000, f1 score: 0.544254, accuracy: 0.568750
K=18 precision: 0.574648, recall: 0.510000, f1 score: 0.540397, accuracy: 0.566250
K=20 precision: 0.583569, recall: 0.515000, f1 score: 0.547145, accuracy: 0.573750


In [58]:
# Lets repeat for K from 25 to 50 as well!
for k in range (25, 51, 5):
    knn_model = KNN(K=k)
    knn_model.fit(x_train,y_train)
    knn_pred = knn_model.predict(x_valid)# NOTE: predict step in KNN is much slower than the train step
    precision, recall, f1, accuracy = get_scores_metrics(knn_pred,y_valid)
    # Print Performance results for each K value
    print("K=%d precision: %f, recall: %f, f1 score: %f, accuracy: %f"%(k,precision,recall,f1, accuracy))

K=25 precision: 0.564246, recall: 0.505000, f1 score: 0.532982, accuracy: 0.557500
K=30 precision: 0.561983, recall: 0.510000, f1 score: 0.534731, accuracy: 0.556250
K=35 precision: 0.544444, recall: 0.490000, f1 score: 0.515789, accuracy: 0.540000
K=40 precision: 0.542614, recall: 0.477500, f1 score: 0.507979, accuracy: 0.537500
K=45 precision: 0.548023, recall: 0.485000, f1 score: 0.514589, accuracy: 0.542500
K=50 precision: 0.546784, recall: 0.467500, f1 score: 0.504043, accuracy: 0.540000


## Question 4).

4. Now instead of having a single multivariate Gaussian distribution per class, each class
is going to be generated by a mixture of 3 Gaussians. For each class, we’ll define
3 Gaussians, with the first Gaussian of the first class sharing the covariance matrix
with the first Gaussian of the second class and so on. For both the classes, fix the
mixture probability as (0.1,0.42,0.48) i.e. the sample has arisen from first Gaussian with
probability 0.1, second with probability 0.42 and so on. Mean for three Gaussians in the
positive class are given as DS2-c1-m1, DS2-c1-m2, DS2-c1-m3. Mean for three Gaussians
in the negative class are gives as DS2-c2-m1, DS2-c2-m2, DS2-c2-m3. Corresponding 3
covariance matrices are given as DS2-cov-1, DS2-cov-2 and DS2-cov-3. Now sample
from this distribution and generate the dataset similar to question 1. Call this dataset
as DS2, and submit it with your code. Follow the instructions from Assignment 1 for
data submission format.

In [29]:
# class 1
DS2_c1_m1 = np.genfromtxt(path+'DS2_c1_m1.txt', delimiter=',')[:-1]
DS2_c1_m2 = np.genfromtxt(path+'DS2_c1_m2.txt', delimiter=',')[:-1]
DS2_c1_m3 = np.genfromtxt(path+'DS2_c1_m3.txt', delimiter=',')[:-1]
# class 2
DS2_c2_m1 = np.genfromtxt(path+'DS2_c2_m1.txt', delimiter=',')[:-1]
DS2_c2_m2 = np.genfromtxt(path+'DS2_c2_m2.txt', delimiter=',')[:-1]
DS2_c2_m3 = np.genfromtxt(path+'DS2_c2_m3.txt', delimiter=',')[:-1]
# class 3
DS2_Cov1 = np.genfromtxt(path+'DS2_Cov1.txt', delimiter=',')[:,:-1]
DS2_Cov2 = np.genfromtxt(path+'DS2_Cov2.txt', delimiter=',')[:,:-1]
DS2_Cov3 = np.genfromtxt(path+'DS2_Cov3.txt', delimiter=',')[:,:-1]

In [30]:
def generate_random_choice(list_of_candidates, number_of_items_to_pick, probability_distribution): 
    return choice(list_of_candidates, number_of_items_to_pick, p=probability_distribution)

In [31]:
data = generate_random_choice([-1,0,1], 2000, [0.1,0.42,0.48])

In [32]:
num_c1 = np.count_nonzero(data == -1)
num_c2 = np.count_nonzero(data == 0)
num_c3 = np.count_nonzero(data == 1)

In [33]:
c1_positive = np.random.multivariate_normal(DS2_c1_m1, DS2_Cov1, num_c1)
c2_positive = np.random.multivariate_normal(DS2_c1_m2, DS2_Cov2, num_c2)
c3_positive = np.random.multivariate_normal(DS2_c1_m3, DS2_Cov3, num_c3)

In [34]:
c1_negative = np.random.multivariate_normal(DS2_c2_m1, DS2_Cov1, num_c1)
c2_negative = np.random.multivariate_normal(DS2_c2_m2, DS2_Cov2, num_c2)
c3_negative = np.random.multivariate_normal(DS2_c2_m3, DS2_Cov3, num_c3)

In [35]:
def append_target_value(class_array, target_array):
    return np.append(class_array,target_array.reshape(-1,1), axis=1)
def append_more_rows(class_array_1, class_array_2):
    return np.append(class_array_1,class_array_2, axis=0)

In [36]:
''' append the target values to the end of the array,
    using the append_target_value function defined in the upper scope'''
class_c1_positive = append_target_value(c1_positive,np.full(num_c1, -1))
class_c2_positive = append_target_value(c2_positive,np.full(num_c2, 0))
class_c3_positive = append_target_value(c3_positive,np.full(num_c3, 1))

In [37]:
''' repeat the append process for the negative class!'''
class_c1_negative = append_target_value(c1_negative,np.full(num_c1, -1))
class_c2_negative = append_target_value(c2_negative,np.full(num_c2, 0))
class_c3_negative = append_target_value(c3_negative,np.full(num_c3, 1))

In [38]:
''' now we can combine all the positive classes together and negative together'''
class_positive = append_more_rows(class_c1_positive,class_c2_positive)
class_positive = append_more_rows(class_positive,class_c3_positive)

In [39]:
''' Repeat for the negative classes'''
class_negative = append_more_rows(class_c1_negative,class_c2_negative)
class_negative = append_more_rows(class_negative,class_c3_negative)

In [40]:
''' now we can split the datasets into train, test and validation in as for question 1'''
class_positive_train, class_positive_test = train_test_split(class_positive, test_size=0.2, random_state=42)
class_positive_train, class_positive_validate = train_test_split(class_positive_train, test_size=0.25, random_state=42)

In [41]:
'''repeat the split steps for negative class'''
class_negative_train, class_negative_test = train_test_split(class_negative, test_size=0.2, random_state=42)
class_negative_train, class_negative_validate = train_test_split(class_negative_train, test_size=0.25, random_state=42)

In [42]:
''' lets combine the dataset now!'''
train_set = append_more_rows(class_positive_train,class_negative_train)
test_set = append_more_rows(class_positive_test,class_negative_test)
validate_set = append_more_rows(class_positive_validate,class_negative_validate)

In [43]:
print(train_set.shape, test_set.shape, validate_set.shape)

(2400, 21) (800, 21) (800, 21)


In [44]:
'''Great every thing looks fine! lets save these dataset then as DS2'''
np.savetxt(path_output+"DS2_train.csv", train_set, delimiter=",")
np.savetxt(path_output+"DS2_test.csv", test_set, delimiter=",")
np.savetxt(path_output+"DS2_validate.csv", validate_set, delimiter=",")

## Question 5

5. Now perform the experiments in questions 2 and 3 again, but now using DS2. 1. Estimate the parameters of the GDA model using the maximum likelihood approach.

    (a) For DS1, report the best fit accuracy, precision, recall and F-measure achieved by the classifier.

    (b) Report the coefficients learnt.

In [45]:
x_train2 = train_set[0:,0:20]
y_train2 = train_set[0:,20:]

x_test2 = test_set[0:,0:20]
y_test2 = test_set[0:,20:]

x_valid2 = validate_set[0:,0:20]
y_valid2 = validate_set[0:,20:]

In [62]:
def get_scores_metrics(gda_pred, y):
    #print(gda_pred)
    accuracy = metrics.accuracy_score(y, gda_pred)          # accuracy: (tp + tn) / (p + n)
    precision = metrics.precision_score(y, gda_pred, average='macro')        # precision tp / (tp + fp)
    recall = metrics.recall_score(y, gda_pred, average='macro')              # recall: tp / (tp + fn)
    f1 = metrics.f1_score(y, gda_pred, average='macro')                      # f1: 2 tp / (2 tp + fp + fn)
    #print(metrics.classification_report(y, gda_pred))
    return precision, recall, f1, accuracy

In [63]:
class GDA:
    def __init__(self,train_data,train_label):
        self.labels, self.class_priors = np.unique(train_label, return_counts=True)
        self.class_priors = self.class_priors / train_label.shape[0]
        
        self.Train_Data = train_data
        self.Train_Label = train_label
        
        print(self.labels)
        
        self.dict_data = {}
        self.dict_data_count = {}
        for (data,label) in zip(self.Train_Data,self.Train_Label):
 
            if label[0] in self.dict_data.keys():
                temp = self.dict_data.get(label[0])
                temp_count = self.dict_data_count.get(label[0])
                temp_count += 1
                temp.append(list(data))
                self.dict_data[label[0]] = temp
                self.dict_data_count[label[0]] = temp_count
            else:
                self.dict_data[label[0]] = [list(data)]
                self.dict_data_count[label[0]] = 1
            
                    
        # Binomial probability goas here:
        row,col = np.shape(train_data)
        self.binomial_dict = {}
        for key, val in self.dict_data_count.items():
            self.binomial_dict[key] = val*1.0/row
        
        dict_data_array = {}
        data_sum_dict = {}
        for k, v in self.dict_data.items():
            dict_data_array[k] = np.array(v)
            data_sum_dict[k]= np.sum(np.array(v),0)
        
        self.mu_data = {}
        for k, v in data_sum_dict.items():
            self.mu_data[k] = v*1.0/self.dict_data_count[k]
            
        dict_data_array2 = {}    
        for k, v in dict_data_array.items():
            dict_data_array2[k] = v-self.mu_data[k]
        
        self.sigma = []
        
        for k,v in dict_data_array2.items():
            for deta in v:
                deta = deta.reshape(1,col)
                ans = deta.T.dot(deta)
                self.sigma.append(ans)

        self.sigma = np.array(self.sigma)
        #print(np.shape(self.sigma))
        self.sigma = np.sum(self.sigma,0)
        self.sigma = self.sigma/row
        
        
        for k, v in self.mu_data.items():
            self.mu_data[k] = v.reshape(1,col)
            print("Mu_{}: {}".format(k,v.reshape(1,col)))
        print(self.sigma)

    def Gaussian(self, x, mean, cov):
        
        dim = np.shape(cov)[0]
        # Cov measures of the determinant is zero
        covdet = np.linalg.det(cov + np.eye(dim) * 0.001)
        covinv = np.linalg.inv(cov + np.eye(dim) * 0.001)
        xdiff = (x - mean).reshape((1, dim))
        # Probability Density
        p_density = 1.0 / (np.power(np.power(2 * np.pi, dim) * np.abs(covdet), 0.5)) * \
               np.exp(-0.5 * xdiff.dot(covinv).dot(xdiff.T))[0][0]
        return p_density

    def predict(self,test_data):
        predict_label = []
        for data in test_data:
            max_label = 0
            max_likehood = 0
            for lab in self.labels:
                likelihood = self.Gaussian(data,self.mu_data[lab],self.sigma)
                if likelihood >= max_likehood:
                    max_likehood = likelihood
                    max_label = lab
            predict_label.append(max_label)
        return predict_label

In [64]:
gda2 = GDA(x_train2,y_train2)

[-1.  0.  1.]
Mu_0.0: [[1.29724275 1.25102015 1.25004288 1.28276062 1.2989058  1.31526389
  1.25354354 1.23550585 1.26595639 1.3037435  1.23173239 1.31843288
  1.36190463 1.28937024 1.29656502 1.34739472 1.37227629 1.29930885
  1.2857059  1.34835781]]
Mu_1.0: [[0.92480382 0.91899941 1.04501533 0.96098354 1.00304952 0.97531307
  0.95780066 0.95276701 0.96189318 0.93158991 0.98127981 1.04586067
  0.95713262 0.98932708 1.02272357 0.97261192 0.98520985 0.98567112
  0.97661387 0.96837571]]
Mu_-1.0: [[1.39698335 1.36873928 1.45029711 1.41682151 1.45625061 1.32099039
  1.41018726 1.42241944 1.45919498 1.35199226 1.37948593 1.32646174
  1.33457397 1.37718355 1.38782479 1.29662418 1.48695119 1.32193448
  1.42819359 1.45026341]]
[[8.00726296 5.60067864 5.02631111 5.25434576 4.5973586  5.93259356
  6.07656546 5.81946632 4.91154092 5.60379888 5.61471706 5.08515782
  5.21811786 6.44090769 5.43450398 5.84128789 5.61719372 5.51110253
  5.72483433 6.04339909]
 [5.60067864 7.36789331 5.16497191 5.34621

In [65]:
pred2 = gda2.predict(x_test2)

In [66]:
precision, recall, f1, accuracy = get_scores_metrics(pred2, y_test2)

In [67]:
#print the results
print('Accuracy: %f' % accuracy)
print('Precision: %f' % precision)
print('Recall: %f' % recall)
print('F1 score: %f' % f1)

Accuracy: 0.348750
Precision: 0.357836
Recall: 0.339353
F1 score: 0.319012


In [70]:
for k in range (2, 21,2):
    knn_model = KNN(K=k)
    knn_model.fit(x_train2,y_train2)
    knn_pred = knn_model.predict(x_test2)# NOTE: predict step in KNN is much slower than the train step
    precision, recall, f1, accuracy = get_scores_metrics(knn_pred, y_test2)
    # Print Performance results for each K value
    print("K=%d precision: %f, recall: %f, f1 score: %f, accuracy: %f"%(k,precision,recall,f1, accuracy))

K=2 precision: 0.765114, recall: 0.732873, f1 score: 0.746064, accuracy: 0.808750
K=4 precision: 0.822172, recall: 0.728152, f1 score: 0.754473, accuracy: 0.833750
K=6 precision: 0.846831, recall: 0.740105, f1 score: 0.769710, accuracy: 0.841250
K=8 precision: 0.859159, recall: 0.694358, f1 score: 0.721160, accuracy: 0.828750
K=10 precision: 0.860319, recall: 0.689397, f1 score: 0.712655, accuracy: 0.835000
K=12 precision: 0.876920, recall: 0.704402, f1 score: 0.733356, accuracy: 0.838750
K=14 precision: 0.875071, recall: 0.686617, f1 score: 0.707264, accuracy: 0.838750
K=16 precision: 0.871801, recall: 0.676856, f1 score: 0.692126, accuracy: 0.837500
K=18 precision: 0.869734, recall: 0.674576, f1 score: 0.690244, accuracy: 0.835000
K=20 precision: 0.886367, recall: 0.657015, f1 score: 0.661999, accuracy: 0.831250


In [71]:
for k in range (25, 51,5):
    knn_model = KNN(K=k)
    knn_model.fit(x_train2,y_train2)
    knn_pred = knn_model.predict(x_test2)# NOTE: predict step in KNN is much slower than the train step
    precision, recall, f1, accuracy = get_scores_metrics(knn_pred, y_test2)
    # Print Performance results for each K value
    print("K=%d precision: %f, recall: %f, f1 score: %f, accuracy: %f"%(k,precision,recall,f1, accuracy))

K=25 precision: 0.881400, recall: 0.648299, f1 score: 0.651022, accuracy: 0.823750
K=30 precision: 0.878603, recall: 0.628865, f1 score: 0.616465, accuracy: 0.817500
K=35 precision: 0.875967, recall: 0.616688, f1 score: 0.593820, accuracy: 0.813750
K=40 precision: 0.872188, recall: 0.612212, f1 score: 0.589453, accuracy: 0.807500
K=45 precision: 0.869721, recall: 0.602404, f1 score: 0.571807, accuracy: 0.802500
K=50 precision: 0.869283, recall: 0.601075, f1 score: 0.570195, accuracy: 0.800000


In [72]:
for k in range (2, 21,2):
    knn_model = KNN(K=k)
    knn_model.fit(x_train2,y_train2)
    knn_pred = knn_model.predict(x_valid2)# NOTE: predict step in KNN is much slower than the train step
    precision, recall, f1, accuracy = get_scores_metrics(knn_pred, y_valid2)
    # Print Performance results for each K value
    print("K=%d precision: %f, recall: %f, f1 score: %f, accuracy: %f"%(k,precision,recall,f1, accuracy))

K=2 precision: 0.784622, recall: 0.745737, f1 score: 0.761230, accuracy: 0.831250
K=4 precision: 0.816232, recall: 0.733402, f1 score: 0.758021, accuracy: 0.848750
K=6 precision: 0.873950, recall: 0.743451, f1 score: 0.775561, accuracy: 0.866250
K=8 precision: 0.869872, recall: 0.717134, f1 score: 0.746390, accuracy: 0.856250
K=10 precision: 0.870350, recall: 0.696943, f1 score: 0.721114, accuracy: 0.850000
K=12 precision: 0.877127, recall: 0.704381, f1 score: 0.728226, accuracy: 0.860000
K=14 precision: 0.884943, recall: 0.691222, f1 score: 0.711258, accuracy: 0.855000
K=16 precision: 0.871573, recall: 0.667044, f1 score: 0.677065, accuracy: 0.843750
K=18 precision: 0.874869, recall: 0.671870, f1 score: 0.684180, accuracy: 0.846250
K=20 precision: 0.873480, recall: 0.666361, f1 score: 0.674148, accuracy: 0.847500


In [73]:
for k in range (25, 51,5):
    knn_model = KNN(K=k)
    knn_model.fit(x_train2,y_train2)
    knn_pred = knn_model.predict(x_valid2)# NOTE: predict step in KNN is much slower than the train step
    precision, recall, f1, accuracy = get_scores_metrics(knn_pred, y_valid2)
    # Print Performance results for each K value
    print("K=%d precision: %f, recall: %f, f1 score: %f, accuracy: %f"%(k,precision,recall,f1, accuracy))

K=25 precision: 0.886115, recall: 0.634610, f1 score: 0.629459, accuracy: 0.826250
K=30 precision: 0.887646, recall: 0.633569, f1 score: 0.624977, accuracy: 0.828750
K=35 precision: 0.885169, recall: 0.627510, f1 score: 0.615490, accuracy: 0.825000
K=40 precision: 0.877862, recall: 0.609737, f1 score: 0.586559, accuracy: 0.813750
K=45 precision: 0.877430, recall: 0.609871, f1 score: 0.586605, accuracy: 0.813750
K=50 precision: 0.875564, recall: 0.602782, f1 score: 0.575803, accuracy: 0.808750
